In [0]:
from google.colab import drive
drive.mount('/content/drive')
dir_path = '/content/drive/My Drive/автобрея/'

Необходимые библиотеки и т.д.

In [0]:
!pip install pymorphy2

In [0]:
import pickle

In [0]:
import pandas as pd
import nltk
nltk.download('stopwords')
import numpy as np
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem
import re

morph = MorphAnalyzer()
token = RegexpTokenizer('\w+')
stops = set(stopwords.words('russian'))

def normalize_pm(text):
    words = [morph.parse(word)[0].normal_form for word in tokenize(text) if word]
    return words

def tokenize(text):
    return token.tokenize(text)

In [0]:
!pip install git+https://github.com/lopuhin/python-adagram.git

In [0]:
!wget https://s3.amazonaws.com/kostia.lopuhin/all.a010.p10.d300.w5.m100.nonorm.slim.joblib -d adagram.joblib

In [0]:
import adagram
vm = adagram.VectorModel.load('all.a010.p10.d300.w5.m100.nonorm.slim.joblib')

In [0]:
from scipy import spatial

In [0]:
import gensim, logging

In [0]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [0]:
amodel = gensim.models.KeyedVectors.load_word2vec_format("/content/drive/My Drive/автобрея/model.bin", binary=True)
amodel.init_sims(replace=True)

In [0]:
## достаем наши сиды и сохраняем в переменные
#### надо будет все это отредачить до просто пикл файла когда будем грузить все на гит (и в другом ноутбуке тоже)
with open("/content/drive/My Drive/автобрея/Food_final_seeds.pickle", "rb") as c: 
  food_seeds = pickle.load(c)

with open("/content/drive/My Drive/автобрея/Service_final_seeds.pickle", "rb") as c:
  service_seeds = pickle.load(c)

In [0]:
## Достаем вектора топиков
with open("/content/drive/My Drive/автобрея/Service_topic_vector.pickle", "rb") as c:
  service_topic_vector = pickle.load(c)

with open("/content/drive/My Drive/автобрея/Food_topic_vector.pickle", "rb") as c:
  food_topic_vector = pickle.load(c)

In [0]:
## достаем векторы негативных и позитивных тональных слов
with open("/content/drive/My Drive/автобрея/Positive_vector.pickle", "rb") as c:
  pos_vector = pickle.load(c)

with open("/content/drive/My Drive/автобрея/Negative_vector.pickle", "rb") as c:
  neg_vector = pickle.load(c)

Классификатор:

In [0]:
#словарь тэгов для конвертации из формата pymorphy в w2v
tags = { 'INFN':'VERB', 'NOUN':'NOUN', 'ADJF':'ADJ', 'ADJS':'ADJ','PREP':'ADP', 'ADVB':'ADV', 'CONJ':'SCONJ', 'PRCL':'PART', 'NUMR':'NUM', 'NPRO':'PRON', 'INTJ':'INTJ', 'PRED':'ADV', 'VERB':'VERB', 'GRND':'GRND', None:'None', 'PRTF':'PRTF', 'PRTS':'PRTS'}

In [0]:
def convert_word(word):                 #в модели вордувек слова хранятся так: дом_NOUN - эта функция преобразует слово в такой формат
    aword = re.sub('ё', "е", word)
    p = morph.parse(word)[0]
    tag = tags[p.tag.POS]
    new_word = aword+'_'+tag
    return new_word

In [0]:
def get_topics(file, service_vector, food_vector):
  food_class_r = {}                             
  service_class_r = {}

  with open(file, 'r', encoding = 'utf-8') as f:
    text = f.read()
    sents = re.split('(?<=[.!?]) (?=[A-ЯЁA-Z])', text)

    for sent in sents:
      for word in sent.split():
        norm = ' '.join(normalize_pm(word))
        if norm not in stops and convert_word(norm) in amodel.wv.vocab:

          word_vector = amodel.wv[convert_word(norm)]
          sim_service = spatial.distance.cosine(word_vector, service_vector)
          sim_food = spatial.distance.cosine(word_vector, food_vector)

          if sim_service<sim_food:
            service_class_r[(norm, sim_service)] = sents.index(sent)
          elif sim_service>sim_food:
            food_class_r[(norm, sim_food)] = sents.index(sent)
  return sents, food_class_r, service_class_r

In [0]:
def classifier1(file, result = set()): ## потом объединяем его со вторым
  with open(file, 'r', encoding = 'utf-8') as f:
    text = f.read()
    sents = re.split('(?<=[.!?]) (?=[A-ЯЁA-Z])', text)
    
  for sent in sents:
    try:
      sent_words = [normalize_pm(w.strip())[0] for w in sent.split()]
    except:
      print('Cannot handle sentence ' + str(sents.index(sent) + 1))
    ## ищем все тональное о еде:
    for line in food_seeds:
      t_word = normalize_pm(line[0].strip())[0]
      tone = bool(int(line[1])) # так удобно их преобразовывать, если встр. "не"
      if t_word in sent_words:
        #id_sent = sents.index(sent) + 1 # прибавляем 1, т.к. в юдпайпе счет с одного
        i = sent_words.index(t_word)
        try:
          if sent_words[i-1] == 'не':
            tone_words = sent_words[i-1] + ' ' + sent_words[i]
            #id_tone = str(i) + ',' + str(i+1) # плюс один по тем же причинам
            tone = int(not tone)
          elif sent_words[i-2] == 'не':
            tone_words = sent_words[i-2] + ' ' + sent_words[i-1] + ' ' + sent_words[i]
            #id_tone = str(i-1) + ',' + str(i) + ',' + str(i+1) # тоже
            tone = int(not tone)
          else:
            tone_words = sent_words[i]
            #id_tone = str(i+1) # тож
            tone = int(tone)
        except IndexError:
          tone_words = sent_words[i]
          tone = int(tone)
        current = 'Food\t' + str(tone) + '\t' + tone_words
        result.add(current)
    ## ищем все тональное о сервисе:
    for line in service_seeds:
      t_word = normalize_pm(line[0].strip())[0]
      tone = bool(int(line[1])) # так удобно их преобразовывать, если встр. "не"
      if t_word in sent_words:
        #id_sent = sents.index(sent) + 1 # прибавляем 1, т.к. в юдпайпе счет с одного
        i = sent_words.index(t_word)
        try:
          if sent_words[i-1] == 'не':
            tone_words = sent_words[i-1] + ' ' + sent_words[i]
            #id_tone = str(i) + ',' + str(i+1) # плюс один по тем же причинам
            tone = int(not tone)
          elif sent_words[i-2] == 'не':
            tone_words = sent_words[i-2] + ' ' + sent_words[i-1] + ' ' + sent_words[i]
            #id_tone = str(i-1) + ',' + str(i) + ',' + str(i+1) # тоже
            tone = int(not tone)
          else:
            tone_words = sent_words[i]
            #id_tone = str(i+1) # тож
            tone = int(tone)
        except IndexError:
          tone_words = sent_words[i]
          tone = int(tone)
        current = 'Service\t' + str(tone) + '\t' + tone_words
        result.add(current)
  return(result)

In [0]:
def vect_sim(word, vector1, vector2):
  sim1 = spatial.distance.cosine(amodel.wv[convert_word(word)], vector1)
  sim2 = spatial.distance.cosine(amodel.wv[convert_word(word)], vector2)
  return sim1, sim2

In [0]:
def classifier2(file, result = set()): 
  """
  Можно вместо пустого резалта давать ему резалт из другого классифаера и они будут объединяться
  Например, можно будет сделать так:
  Сверху у нас уже есть simple_result первого классификатора на тексте 6668
  Можно запустить классификатор два на этом результате
  result = classifier2(file, result = simple_result)
  Тогда классификатор2 будет добавлять свои штучки в то же множество, и данные не будут повторяться

  И вообще потом эти функции можно запускать на куче текстов друг за другом
  """
  sents, food_class_r, service_class_r = get_topics(file, service_topic_vector, food_topic_vector)

  for obj in food_class_r:
    id_sent = food_class_r[obj]
    sent = re.sub ('- ', '', sents[id_sent])
    sent_words = [normalize_pm(w.strip())[0] for w in sent.split()]
    asp_label = 'Food'
    word = obj[0]
    p = morph.parse(word)[0]
    pos = p.tag.POS
    id_word = sent_words.index(word)

    if pos == 'ADVB':
      sim_pos, sim_neg = vect_sim(word, pos_vector, neg_vector)
      if sim_pos < sim_neg:
        label = 1
      else: 
        label = 0
      current = str(asp_label) + '\t' + str(label) + '\t' + word
      result.add(current)
      
    elif pos == 'ADJF':
      try:
        target_noun1 = sent_words[id_word+1]
        p = morph.parse(target_noun1)[0]
        pos1 = p.tag.POS 
      except IndexError:
        pass
      try:
        target_noun2 = sent_words[id_word-2]
        p = morph.parse(target_noun2)[0]
        pos2 = p.tag.POS
      except IndexError:
        pass

      if pos1 and pos1 == 'NOUN':
        sim_service, sim_food = vect_sim(target_noun1, service_topic_vector, food_topic_vector)
        if sim_service < 0.5 or sim_food < 0.5: #не берем совсем неблизкие к фуд и сервис штуки
          if sim_service < sim_food:
            asp_label = 'Service'
          else:
            asp_label = 'Food'
        else:
          asp_label = None
        sim_pos, sim_neg = vect_sim(word, pos_vector, neg_vector)
        if sim_pos < sim_neg:
          label = 1
        else: 
          label = 0
      elif pos2 and pos2 == 'NOUN':
        sim_service, sim_food = vect_sim(target_noun2, service_topic_vector, food_topic_vector)
        if sim_service < 0.5 or sim_food < 0.5: #не берем совсем неблизкие к фуд и сервис штуки
          if sim_service < sim_food:
            asp_label = 'Service'
          else:
            asp_label = 'Food'
        else:
          asp_label = None
        sim_pos, sim_neg = vect_sim(word, pos_vector, neg_vector)
        if sim_pos < sim_neg:
          label = 1
        else: 
          label = 0
      else:
        label = None
        asp_label = None
      if abs(sim_pos - sim_neg) < 0.02:
        label = None 

      current = str(asp_label) + '\t' + str(label) + '\t' + word
      result.add(current)
      #print(current, word)
    
    if pos == 'NOUN':
      #adj = None
      pos1 = None
      pos2 = None
      try:
        target_adj1 = sent_words[id_word-1]
        p = morph.parse(target_adj1)[0]
        pos1 = p.tag.POS
        
      except IndexError:
        pass
      try:
        target_adj2 = sent_words[id_word+2]
        p = morph.parse(target_adj2)[0]
        pos2 = p.tag.POS
      except IndexError:
        pass
      if pos1 and pos1 == 'ADJF':
        adj = target_adj1
        sim_pos, sim_neg = vect_sim(adj, pos_vector, neg_vector)
        if sim_pos < sim_neg:
          label = 1
        else: 
          label = 0
        if sent_words[sent_words.index(adj)-1] == 'не':
          if label == 1:
            label == 0
          elif label == 0:
            label = 1
          tone_words = 'не ' + adj
          #id_word = str(sent_words.index(adj)) + ',' + str(sent_words.index(adj) + 1) #тк в юдпайпе не с нуля
        else:
          tone_words = adj
          #id_word = str(sent_words.index(adj) + 1)

      elif pos2 and pos2 == 'ADJF':
        adj = target_adj2
        sim_pos, sim_neg = vect_sim(adj, pos_vector, neg_vector)
        if sim_pos < sim_neg:
          label = 1
        else: 
          label = 0
        if sent_words[sent_words.index(adj)-1] == 'не':
          if label == 1:
            label == 0
          elif label == 0:
            label = 1
          tone_words = 'не ' + adj
          #id_word = str(sent_words.index(adj)) + ',' + str(sent_words.index(adj) + 1) #тк в юдпайпе не с нуля
        else: 
          tone_words = adj
          #id_word = str(sent_words.index(adj) + 1)

      else:
        label = None
        tone_words = None
      if abs(sim_pos - sim_neg) < 0.02:
        label = None 
      current = str(asp_label) + '\t' + str(label) + '\t' + str(tone_words)
      result.add(current)
      #print(current, word)

  final_result = set()
  for i in result:
    if "None" not in i:
      final_result.add(i)

  return final_result

In [0]:
# вот эту функцию можно применять на любом тексте и получать тональный словарь текста в переменной и файлом
def final_classifier(file, result = set()):
  middle_result = classifier1(file, result)
  final_result = classifier2(file, middle_result)

  new_file = file.split('.')[0] + '_done.txt'
  with open(new_file, 'a', encoding = 'utf-8') as w:
    for i in final_result:
      w.write(i + '\n')
  return final_result

In [0]:
#for file in files:
#  final_classifier(file)

Ниже просто всякие проверки

In [0]:
test_file = '/content/drive/My Drive/автобрея/6668.txt'

In [0]:
simple_result = classifier1(test_file)

Cannot handle sentence 11


In [0]:
simple_result

{'Food\t0\tплохой',
 'Food\t1\tбольшой',
 'Food\t1\tвкусно',
 'Food\t1\tвкусный',
 'Food\t1\tзамечательный',
 'Food\t1\tнеплохой',
 'Food\t1\tпонравиться',
 'Food\t1\tприятный',
 'Food\t1\tсытный',
 'Food\t1\tхороший',
 'Service\t0\tплохой',
 'Service\t1\tзамечательный',
 'Service\t1\tнедорогой',
 'Service\t1\tнеплохой',
 'Service\t1\tпонравиться',
 'Service\t1\tприятный',
 'Service\t1\tрадушно',
 'Service\t1\tспасибо',
 'Service\t1\tхороший'}

In [0]:
classifier2(test_file)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


{'Food\t0\tпоздний',
 'Food\t1\tвкусно',
 'Food\t1\tне огромный',
 'Food\t1\tсытный'}

In [0]:
result_6668 = classifier2(test_file, simple_result)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
result_6668

{'Food\t0\tплохой',
 'Food\t0\tпоздний',
 'Food\t1\tбольшой',
 'Food\t1\tвкусно',
 'Food\t1\tвкусный',
 'Food\t1\tзамечательный',
 'Food\t1\tне огромный',
 'Food\t1\tнеплохой',
 'Food\t1\tпонравиться',
 'Food\t1\tприятный',
 'Food\t1\tсытный',
 'Food\t1\tхороший',
 'Service\t0\tплохой',
 'Service\t1\tзамечательный',
 'Service\t1\tнедорогой',
 'Service\t1\tнеплохой',
 'Service\t1\tпонравиться',
 'Service\t1\tприятный',
 'Service\t1\tрадушно',
 'Service\t1\tспасибо',
 'Service\t1\tхороший'}

In [0]:
len(result_6668)

21

In [0]:
result_6668_2 = final_classifier(test_file)

Cannot handle sentence 11


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
len(result_6668_2)

21